# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,11.57,95,100,1.34,NZ,1689975248
1,1,grytviken,-54.2811,-36.5092,3.18,97,100,6.62,GS,1689975248
2,2,thompson,55.7435,-97.8558,20.09,77,75,3.60,CA,1689975248
3,3,longyearbyen,78.2186,15.6401,6.91,81,100,5.66,SJ,1689975249
4,4,redencao,-4.2258,-38.7306,22.77,76,16,1.26,BR,1689975249


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)


# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Define your ideal weather condition criteria
ideal_conditions = (city_data_df['Max Temp'] >= 21) & (city_data_df['Max Temp'] <= 27) & \
                   (city_data_df['Humidity'] < 100) & \
                   (city_data_df['Cloudiness'] == 0) & \
                   (city_data_df['Wind Speed'] <= 5)

# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[ideal_conditions]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna().reset_index(drop=True)

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,91,odessa,46.4775,30.7326,23.34,59,0,3.35,UA,1689975265
1,113,astrakhan,46.3497,48.0408,23.93,78,0,2.00,RU,1689975274
2,151,maloye ganyushkino,46.6000,49.2667,25.63,56,0,2.67,KZ,1689975288
3,171,kitob,39.0843,66.8332,26.62,26,0,4.34,UZ,1689975292
4,192,pisco,-13.7000,-76.2167,21.03,73,0,4.63,PE,1689975298
5,223,porto franco,-6.3383,-47.3992,26.30,27,0,2.10,BR,1689975306
6,230,marivan,35.5219,46.1760,23.68,18,0,0.91,IR,1689975308
7,328,fort mcmurray,56.7268,-111.3810,26.77,50,0,2.57,CA,1689975056
8,378,jardim,-21.4803,-56.1381,25.43,41,0,1.99,BR,1689975343
9,380,karatau,43.1667,70.4667,24.15,27,0,2.58,KZ,1689975343


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,odessa,UA,46.4775,30.7326,59,
1,astrakhan,RU,46.3497,48.0408,78,
2,maloye ganyushkino,KZ,46.6000,49.2667,56,
3,kitob,UZ,39.0843,66.8332,26,
4,pisco,PE,-13.7000,-76.2167,73,
5,porto franco,BR,-6.3383,-47.3992,27,
6,marivan,IR,35.5219,46.1760,18,
7,fort mcmurray,CA,56.7268,-111.3810,50,
8,jardim,BR,-21.4803,-56.1381,41,
9,karatau,KZ,43.1667,70.4667,27,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000  #10,000 meters
params = {
    "type": "accommodation.hotel",
    "format":"json",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
odessa - nearest hotel: Готель Венера
astrakhan - nearest hotel: Omega
maloye ganyushkino - nearest hotel: Конил Сазы
kitob - nearest hotel: Shakhrisabz Yulduzi
pisco - nearest hotel: La Portada
porto franco - nearest hotel: Hotel Belém-Brasilia
marivan - nearest hotel: هتل میران
fort mcmurray - nearest hotel: Nomad Hotel and Suites
jardim - nearest hotel: Hotel Uchôa
karatau - nearest hotel: Артабас
parauapebas - nearest hotel: Hotel Igarapé
alexandria - nearest hotel: Smoha Zahran Haus (7th floor)
saham al kaffarat - nearest hotel: العشم
bataipora - nearest hotel: Hotel Estoril


,City,Country,Lat,Lng,Humidity,Hotel Name
0,odessa,UA,46.4775,30.7326,59,Готель Венера
1,astrakhan,RU,46.3497,48.0408,78,Omega
2,maloye ganyushkino,KZ,46.6000,49.2667,56,Конил Сазы
3,kitob,UZ,39.0843,66.8332,26,Shakhrisabz Yulduzi
4,pisco,PE,-13.7000,-76.2167,73,La Portada
5,porto franco,BR,-6.3383,-47.3992,27,Hotel Belém-Brasilia
6,marivan,IR,35.5219,46.1760,18,هتل میران
7,fort mcmurray,CA,56.7268,-111.3810,50,Nomad Hotel and Suites
8,jardim,BR,-21.4803,-56.1381,41,Hotel Uchôa
9,karatau,KZ,43.1667,70.4667,27,Артабас


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_1 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_height = 600,
    size = "Humidity",
    color = "City",
    hover_cols = ["City", "Country", "Humidity", "Hotel Name"]  # Specify the columns to include in the hover tooltip
)

# Display the map with hover tooltips
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)